In [2]:
import Pkg
Pkg.activate("..")

  Activating project at `~/Projects/comp441/latentplan.jl`


In [6]:
# Set display width, load packages
ENV["COLUMNS"]=72
using Knet, MLDatasets, IterTools

In [7]:
# Define convolutional layer:
struct Conv; w; b; f; end
(c::Conv)(x) = c.f.(pool(conv4(c.w, x) .+ c.b))
Conv(w1,w2,cx,cy,f=relu) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f);

In [8]:
# Define dense layer:
struct Dense; w; b; f; end
(d::Dense)(x) = d.f.(d.w * mat(x) .+ d.b)
Dense(i::Int,o::Int,f=relu) = Dense(param(o,i), param0(o), f);

In [9]:
# Define a chain of layers:
struct Chain; layers; Chain(args...)=new(args); end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)

In [10]:
# Load MNIST data
xtrn,ytrn = MNIST.traindata(Float32); ytrn[ytrn.==0] .= 10
xtst,ytst = MNIST.testdata(Float32);  ytst[ytst.==0] .= 10
dtrn = minibatch(xtrn, ytrn, 100; xsize=(size(xtrn,1),size(xtrn,2),1,:))
dtst = minibatch(xtst, ytst, 100; xsize=(size(xtst,1),size(xtst,2),1,:));

This program has requested access to the data dependency MNIST.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Dataset: THE MNIST DATABASE of handwritten digits
Authors: Yann LeCun, Corinna Cortes, Christopher J.C. Burges
Website: http://yann.lecun.com/exdb/mnist/

[LeCun et al., 1998a]
    Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner.
    "Gradient-based learning applied to document recognition."
    Proceedings of the IEEE, 86(11):2278-2324, November 1998

The files are available for download at the offical
website linked above. Note that using the data
responsibly and respecting copyright remains your
responsibility. The authors of MNIST aren't really
explicit about any terms of use, so please read the
website to make sure you want to download the
dataset.



Do you want to download the dataset from ["https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz", "https://ossci-datasets.s3.amazonaws.com/mn

stdin>  y


In [29]:
for data in ncycle(dtrn, 10)
    x = data[1]
    println(size(x))
    x = l1(x)
    println(size(x))
    x = l2(x)
    println(size(x))
    x = l3(x)
    println(size(x))
    x = l4(x)
    println(size(x))
    break
end

(28, 28, 1, 100)
(12, 12, 20, 100)
(4, 4, 50, 100)
(500, 100)
(10, 100)


In [22]:
l1 = Conv(5,5,1,20)
l2 = Conv(5,5,20,50)
l3 = Dense(800,500)
l4 = Dense(500,10,identity)

Dense(P(Matrix{Float32}(10,500)), P(Vector{Float32}(10)), identity)

In [ ]:
# Train and test LeNet (about 30 secs on a gpu to reach 99% accuracy)

progress!(adam(LeNet, ncycle(dtrn,10)))
accuracy(LeNet, dtst)